In [1]:
import json
import torch.nn as nn
import torch
from transformers import BertTokenizer, BertModel, GPT2LMHeadModel, GPT2Tokenizer, GPT2Model
import torch.utils.data as Data
from tqdm import tqdm
import torch.nn as nn

def ranked_sents_to_pairs(pth, tokenizer, special='[SEP]', MAX_LENGTH=256, diffusion=0.24, p1=0.3, p2=0.1, p3=0.2):
    data=[]
    with open(pth, encoding='utf-8') as f:
        samples=[json.loads(sample) for sample in f.readlines()]
    embeds_a=[]
    embeds_b=[]
    embeds_c=[]
    embeds_d=[]
    for sample in samples[:-20]:
        d=[sample['content']+' '+special+' '+sent for sent in sample['results']]
        window_size=len(d)//2
        for i in range(len(d)-window_size):
            if random.random()<diffusion:
                sent=call_function_with_probability(d[i], p1=p1, p2=p2, p3=p3)
            else:
                sent=d[i]
            embeds_a.append(tokenizer.encode(sent, return_tensors='pt', padding='max_length', \
                                    truncation=True, add_special_tokens=True, max_length=MAX_LENGTH))
            if random.random()<diffusion:
                sent=call_function_with_probability(d[i+window_size], p1=p1, p2=p2, p3=p3)
            else:
                sent=d[i+window_size]
            embeds_b.append(tokenizer.encode(d[i+window_size], return_tensors='pt', padding='max_length', \
                                    truncation=True, add_special_tokens=True, max_length=MAX_LENGTH))
    for sample in samples[-100:]:
        d=[sample['content']+' '+special+' '+sent for sent in sample['results']]
        window_size=len(d)//2
        for i in range(len(d)-window_size):
            if random.random()<diffusion:
                sent=call_function_with_probability(d[i], p1=p1, p2=p2, p3=p3)
            else:
                sent=d[i]
            embeds_c.append(tokenizer.encode(d[i], return_tensors='pt', padding='max_length', \
                                    truncation=True, add_special_tokens=True, max_length=MAX_LENGTH))
            if random.random()<diffusion:
                sent=call_function_with_probability(d[i+window_size], p1=p1, p2=p2, p3=p3)
            else:
                sent=d[i+window_size]
            embeds_d.append(tokenizer.encode(d[i+window_size], return_tensors='pt', padding='max_length', \
                                    truncation=True, add_special_tokens=True, max_length=MAX_LENGTH))        
    return torch.cat(embeds_a, 0), torch.cat(embeds_b ,0), torch.cat(embeds_c, 0), torch.cat(embeds_d ,0)

def reward_mdel_loss(a, b):
    return sum(-torch.log(torch.sigmoid(a-b)))/a.size(-1)

class MLP(nn.Module):
    def __init__(self, vocab_size, p=0.1, relu=0.05):
        super(MLP, self).__init__()
        self.bert=GPT2Model.from_pretrained('gpt2')
        self.bert.resize_token_embeddings(vocab_size)
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(p=p)
        self.leaky_relu = nn.LeakyReLU(relu)
    def forward(self, x):
        x=self.bert(x)[0][:,-1:]
        x = self.fc1(x)
        x = self.leaky_relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.leaky_relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    
def model_eval(test_data_iter, model, device):
    model.eval()
    total=0
    acc=0
    for a, b in test_data_iter:
        opt_a=model(a.to(device))
        opt_b=model(b.to(device))
        opt=opt_a.view(-1)-opt_b.view(-1)
        for i in opt.view(-1):
            if float(i)>0:
                acc+=1
            total+=1
    print('Acc: ', round(acc/total*100, 3), '%')
    return

def train_reward_model(data_iter, test_data_iter, mlp, lr=0.0001, epoch_num=50, loss_print_step=10, eval_step=200, clip=50):
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mlp.train()
    '''
    pre_lm=pre_lm.to(device)
    pre_lm.train()
    '''
    #pre_lm.eval()
    optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
    iter_count=0
    print_losses=[]
    for epoch in range(epoch_num):
        print('--------- Epoch: ', epoch, ' ---------')
        for data_a, data_b in tqdm(data_iter):
            data_a=data_a.to(device)
            data_b=data_b.to(device)
            optimizer.zero_grad()
            iter_count+=1
            #data_a=pre_lm(data_a)[1]
            #data_b=pre_lm(data_b)[1]
            data_a=mlp(data_a)
            data_b=mlp(data_b)
            loss=reward_mdel_loss(data_a.view(-1), data_b.view(-1))
            print_losses.append(loss.item())
            loss.backward()
            _ = nn.utils.clip_grad_norm_(mlp.parameters(), clip)
            optimizer.step()
            if iter_count%loss_print_step==0:
                print('Training Loss:',round(float(sum(print_losses) / loss_print_step), 3))
                print_losses=[]
            
            if iter_count%eval_step==0:
                model_eval(test_data_iter, mlp, device=device)
                mlp.train()
            
        torch.save(mlp.state_dict(), './results/mlp'+str(iter_count)+'.pth')

In [2]:
# 可被py文件取代
from nltk.tokenize import sent_tokenize, word_tokenize
import random
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

def remove_negations(sentence):
    # 定义否定词列表
    negations = ['not', 'no', 'never', 'none', 'neither', 'nor', 'nobody', 'nothing', 'nowhere']
    # 分词
    words = word_tokenize(sentence)
    # 遍历单词列表
    new_words = []
    i = 0
    while i < len(words):
        # 如果当前单词为否定词
        if words[i].lower() in negations:
            # 删除当前单词和下一个单词（如果有）
            del words[i:i+2]
        else:
            # 将当前单词添加到新列表中
            new_words.append(words[i])
            i += 1
    # 重新组合单词，生成新的句子
    new_sentence = ' '.join(new_words)
    return new_sentence

def add_negations(sentence):
    # 定义可用于添加否定的单词列表
    negatable_words = ['like', 'enjoy', 'love', 'appreciate', 'want', 'need', 'desire', 'crave', 'hope']
    # 分词
    words = word_tokenize(sentence)
    # 遍历单词列表
    new_words = []
    i = 0
    while i < len(words):
        # 如果当前单词可以被否定
        if words[i].lower() in negatable_words:
            # 如果下一个单词不是否定词，则添加否定词"not"
            if i < len(words) - 1 and words[i+1].lower() not in ['not', 'no', 'never']:
                new_words.extend(['not', words[i]])
                i += 1
            # 否则，不添加否定词
            else:
                new_words.append(words[i])
        else:
            new_words.append(words[i])
        i += 1
    # 重新组合单词，生成新的句子
    new_sentence = ' '.join(new_words)
    return new_sentence

def modify_sentence(sentence, new_word):
    words = sentence.split()
    index = random.randint(0, len(words)-1) # 随机选择一个单词的下标
    operation = random.randint(0, 2) # 随机选择一种操作

    if operation == 0: # 删除随机单词
        del words[index]
    elif operation == 1: # 在随机位置插入新单词
        words.insert(index, new_word)
    else: # 将一个随机单词替换成新单词
        words[index] = new_word
    new_sentence = ' '.join(words)
    return new_sentence

def shuffle_sentences(article):
    sentences = article.split('.')
    sentences = [s.strip() for s in sentences if s.strip()] # 去除空句子
    random.shuffle(sentences)
    shuffled_article = '. '.join(sentences) + '.'
    return shuffled_article

def sentence_shuffle(sentence):
    words = sentence.split()
    if random.random() < 0.5:
        # swap two random words
        i, j = random.sample(range(len(words)), 2)
        words[i], words[j] = words[j], words[i]
    else:
        # move one random word to another random position
        i, j = random.sample(range(len(words)), 2)
        words.insert(j, words.pop(i))
    return ' '.join(words)

def negation_operations(sentence, p1=0.5):
    p = random.random()
    if p < p1:
        return remove_negations(sentence)
    else:
        return add_negations(sentence)   
    
def call_function_with_probability(sentence, p1=0.3, p2=0.1, p3=0.2):   
    #这个是加噪，p1, p2, p3 分别是编辑操作，添加否定词，打乱句子顺序和打乱单词顺序
    p = random.random()
    if p < p1:
        new_word=tokenizer.decode([random.randint(999, tokenizer.vocab_size)])
        return modify_sentence(sentence, new_word=new_word)
    elif p < p1 + p2:
        return negation_operations(sentence)
    elif p < p1 + p2 + p3:
        return shuffle_sentences(sentence)    
    else:
        return sentence_shuffle(sentence)

In [4]:
MAX_LENGTH=256
BATCH_SIZE=16
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Data loading...')
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.add_special_tokens({'sep_token': '__eou__'})
tokenizer.add_special_tokens({'sep_token': '[SEP]'})
#dataset/CornellMovie-Dialog/generated_text_results.json
data_a, data_b, data_c, data_d=ranked_sents_to_pairs('C:/Users/DELL/reward_model_ys/dataset/CornellMovie-Dialog/generated_text_results.json',tokenizer, diffusion=0.24, MAX_LENGTH=MAX_LENGTH)
dataset=Data.TensorDataset(data_a, data_b)
data_iter=torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset=Data.TensorDataset(data_c, data_d)
test_data_iter=torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

Data loading...


In [5]:
mlp=MLP(len(tokenizer)).to(device)

In [ ]:
print('Training...')
train_reward_model(data_iter, test_data_iter, mlp, lr=0.00003, epoch_num=50, loss_print_step=10, eval_step=50)

Training...
--------- Epoch:  0  ---------


  3%|██                                                                               | 10/388 [00:09<04:47,  1.31it/s]

Training Loss: 0.772


  5%|████▏                                                                            | 20/388 [00:17<04:30,  1.36it/s]

Training Loss: 0.802


...

  4%|███▎                                                                             | 16/388 [00:12<04:50,  1.28it/s]

Training Loss: 0.103


  7%|█████▍                                                                           | 26/388 [00:20<04:40,  1.29it/s]

Training Loss: 0.105


  9%|███████▎                                                                         | 35/388 [00:27<04:34,  1.29it/s]